In [ ]:
import math
import agentpy as ap
import numpy as np
# Visualization
import seaborn as sns
import pandas as pd
import random
import matplotlib.pyplot as plt
import IPython
from IPython.display import display, HTML

In [ ]:
class InitialPosition():
    """
    Initial_Position
    """
    
    def __init__(self,agents=None,board_dimensions=None,box_positions=None,discharge_positions=None):
        self.agents = agents
        self.board_dimensions = board_dimensions
        self.box_positions = box_positions
        self.discharge_positions = discharge_positions
    
    def __str__(self):
        """
        The __str__ function is called when the class is converted to a string.
        It returns a string representation of the InitialPosition.
        """
        return f"\n\
        agents: {self.agents}, \n\
        board_dimensions: {self.board_dimensions}, \n\
        box_positions: {self.box_positions}, \n\
        discharge_positions: {self.discharge_positions}"


In [ ]:
class Move():
    rounds = []
    curr_round = -1
    def __init__(self,agent_id=None,action=None,cell=None,looking_direction=None):
        self.agent_id = agent_id
        self.action = action
        self.cell = cell
        self.looking_direction = looking_direction
    
    def __str__(self):
        """
        The __str__ function is called when the class is converted to a string.
        It returns a string representation of the move.
        """
        return f"\n\
        Agent_Id: {self.agent_id}, \n\
        Action: {self.action}, \n\
        Cell: {self.cell}, \n\
        Looking_Direction: {self.looking_direction}"
    
    def add_move(self):
        Move.rounds[Move.curr_round].append(self)
    
    @classmethod
    def start_new_round(cls):
        cls.curr_round += 1
        cls.rounds.append([])
    
    @classmethod
    def start_new_game(cls):
        cls.curr_round = -1
        cls.rounds = []

In [ ]:
class RandomAgent(ap.Agent):
    """ 
    Se mueve a celdas al azar
    Cuando se encuentre una celda con una sola caja, la recoge
    Si esta cargando una caja y ve otra caja, las apila
    """
    
    
    def setup(self):
        self.intention = None  # Intención actual (pick_up, stack, move o wait)
        self.intention_position = None  # Posicion bjetivo al hacer accion
        
        self.nearby_single_boxes = []  # Coordenadas de las cajas cercanas, en stacks de 1 sola caja
        self.nearby_box_stack = []  # Coordenadas de stacks de cajas cercanos. Stacks de mas de 1 caja
        self.has_box = False
        
        self.looking_direction = "Up"  # To facilitate later designing a simulation
        
    def setup_position(self, start_position):
        self.position = start_position # Posición del agente (x, y)
        
    def see(self):
        """ Percibe su entorno """
        x, y = self.position
        self.nearby_single_boxes = []
        self.nearby_box_stack = []
        
        neighbors = self.model.get_neighbors(x, y) 
        for neighbor_pos in neighbors:
            i, j = neighbor_pos
            if not self.model.is_valid_position(neighbor_pos):
                continue
            
            if self.model.box_count[i][j] == 1:
                self.nearby_single_boxes.append(neighbor_pos)
            
            if 1 <= self.model.box_count[i][j] <= 4: # Make sure no more than 5!
                self.nearby_box_stack.append(neighbor_pos)  # Stacks de 1 cuentan para iniciar un stack
    
    def next(self):
        """ Decide la próxima acción según lo que percibe """
        self.intention = "wait" # por defecto espera
        
        temporary_intention = "wait"
        if not self.has_box and len(self.nearby_single_boxes) > 0:
            temporary_intention = "pick_up"
            self.intention_position = self.model.nprandom.choice(self.nearby_single_boxes)
        elif self.has_box and len(self.nearby_box_stack) > 0:
            temporary_intention = "stack"
            self.intention_position = self.model.nprandom.choice(self.nearby_box_stack)
        else:
            temporary_intention = "move"
            self.intention_position = self.model.random.choice(
                self.model.get_neighbors(self.position[0], self.position[1])
            )
            
        if self.model.is_valid_position(self.intention_position):
            self.intention = temporary_intention
                
    def action(self):
        """ Ejecuta la accion """
        x, y = self.intention_position
        if self.intention == "stack":
            self.stack_boxes(x, y)
        elif self.intention == "pick_up":
            self.pick_box(x, y)
        elif self.intention == "move" and self.model.is_valid_move_position(self.intention_position):
            self.position = self.intention_position
            
    def move(self):
        if not self.model.is_floor_organized:
            self.model.total_moves += 1
        self.see()
        self.next()
        self.action()
        # saving move to  move_stack
        self.model.record_move(self)
    
    """ Aux functions """
    # Assumes that given position is valid
    def pick_box(self, x, y):
        self.updateLookingDirection(x, y)
        if self.model.update_box_count(x, y, -1):
            self.has_box = 1
        else:
            self.intention = "wait"
    
    # Assumes that given position is valid
    def stack_boxes(self, x, y): 
        self.updateLookingDirection(x, y)
        if self.model.update_box_count(x, y, 1):
            self.has_box = 0
        else:
            self.intention = "wait"
    def updateLookingDirection(self, to_x, to_y):
        from_x, from_y = self.position
        
        # TODO: Verify which side is up in simulation. Rn positive x is down, positive y is right
        if to_x > from_x:
            self.looking_direction = "down"
        elif to_x < from_x:
            self.looking_direction = "up"
        elif to_y > from_y:
            self.looking_direction = "right"
        elif to_y < from_y:
            self.looking_direction = "left"  


In [ ]:
class RodrigoAgent(ap.Agent):
    """
        Este agente tiene 3 tipos de modos de juego
        
        El primero es cuando tiene una caja, aqui busca encontrar el camino mas corto la discharge_location 
        mas cercana al centro a la que no le consta que no esta llena o es inalcanzable. Esto lo hace con BFS
        sobre el internal_board que es un tablero donde nosotros construimos informacion del tablero real basado
        en lo que hemos visto.
        
        El segundo es cuando no tiene una caja pero recuerda haber visto una caja en algun lado, aqui encuentra el camino
        mas corto a esta caja y la recoge.
        
        El tercero es cuando no recuerda el lugar de ninguna caja ni tiene ninguna caja en mano,
        aqui lo que hace es que visita la casilla mas cercana que todavia no visita, asi buscando explorar de
        manera eficiente.
    """
    
    def setup(self):
        self.intention = None  # Intención actual (pick_up, stack, move o wait)
        self.intention_position = None  # Posición objetivo al hacer acción
        
        self.nearby_single_boxes = []  # Coordenadas de las cajas cercanas (stacks de 1 sola caja)
        self.nearby_box_stack = []  # Coordenadas de stacks de cajas cercanos (stacks de más de 1 caja)
        self.has_box = False
        
        self.looking_direction = "Up"  # Dirección en la que está mirando
        
        # Inicializar tablero interno
        self.internal_board = [
            [0 for _ in range(self.model.p.m)] for _ in range(self.model.p.n)
        ]
        self.visited_cells = [
            [False for _ in range(self.model.p.m)] for _ in range(self.model.p.n)
        ]
        
        self.bfs_path = []  # Lista de movimientos para el BFS
        self.unvisited_path = [] # Lista de movimientos para visitar a la celda no visitada mas cercana a nosotros
        self.seen_box_path = []
        self.seen_box_stack = [] # Lista de Cajas que he visto pero no he podido recojer
        self.discharge_locations = self.model.discharge_locations
        self.discharge_location = (-1, -1) # al principio no hay ninguna seleccionada

    def setup_position(self, start_position):
        self.position = start_position  # Posición inicial del agente
        self.visited_cells[start_position[0]][start_position[1]] = True
        
    def see(self):
        """ Percibe su entorno y actualiza el internal_board """
        x, y = self.position
        self.nearby_single_boxes = []
        self.nearby_box_stack = []
        
        neighbors = self.model.get_neighbors(x, y)
        for neighbor_pos in neighbors:
            i, j = neighbor_pos
            if not self.model.is_valid_position(neighbor_pos):
                continue
            self.visited_cells[i][j] = True
            box_count = self.model.box_count[i][j]
            self.internal_board[i][j] = box_count
            
            if box_count == 0 and (i, j) in self.seen_box_stack:
                self.seen_box_stack.remove((i, j))
                
            if box_count != 0 and (i, j) not in self.seen_box_stack and (i, j) not in self.discharge_locations:
                self.seen_box_stack.append((i, j))
                
            if box_count == 1 and (i, j) not in self.discharge_locations:
                self.nearby_single_boxes.append(neighbor_pos)
            elif 1 <= box_count <= 4:
                self.nearby_box_stack.append(neighbor_pos)
    
    def choose_plan(self):
        """ Escoge el plan según el estado actual """
        if self.has_box:
            return "find_discharge_location"
        else:
            return "explore"
    
    def find_discharge_location(self):
        """ Encuentra la ruta a una discharge_location usando BFS """
        # Vemos si alguno de los vecinos es la discharge_location que buscamos
        for neighbor in self.model.get_neighbors(*self.position):
            
            if (int(neighbor[0]), int(neighbor[1])) == (int(self.discharge_location[0]), int(self.discharge_location[1])):
                if self.model.box_count[neighbor[0]][neighbor[1]] < 5:
                    self.intention = "stack"
                    self.intention_position = neighbor
                    
                    # checamos si hay un agente en la posicion donde queremos poner una caja que es adjacente a nosotros  
                    for agent in self.model.all_agents:
                        if np.array_equal(agent.position, self.intention_position):  # Si pasa hacemos un movimiento random
                            self.intention = "move"
                            neighbors = self.model.get_neighbors(self.position[0], self.position[1])
                            self.intention_position = self.model.nprandom.choice(neighbors)
                    self.bfs_path = []
                    return
                else:
                    self.bfs_path = []
            

        # Si no hemos computado el mejor camino al centro lo encontramos con bfs
        if not self.bfs_path:
            start = self.position
            # Calculamos el centro del tablero
            center_x, center_y = self.p.n / 2, self.p.m / 2

            # Filtramos las locaciones invalidas
            valid_discharge_locations = [
                loc for loc in self.discharge_locations
                if self.internal_board[loc[0]][loc[1]] < 5  
            ]

            # Seleccionamos la mas cercana al centro
            if valid_discharge_locations:
                self.discharge_location = min(
                    valid_discharge_locations,
                    key=lambda loc: ((loc[0] - center_x) ** 2 + (loc[1] - center_y) ** 2) ** 0.5
                )
            else:
                return
            # Encontramos el mejor camino
            self.bfs_path = self.bfs(start, self.discharge_location)
            if self.bfs_path:
                self.bfs_path.pop(0)
        
        if self.bfs_path:
            # Nos intentamos mover a la posicion siguiente
            self.intention_position = self.bfs_path.pop(0)
            x, y = self.intention_position
            self.intention = "move"
            if not self.model.is_valid_move_position(self.intention_position):
                # Hacemos movimiento random
                self.intention = "move"
                neighbors = self.model.get_neighbors(self.position[0], self.position[1])
                self.intention_position = self.model.nprandom.choice(neighbors)
                self.bfs_path = []
        else:
            self.intention = "move"
            neighbors = self.model.get_neighbors(self.position[0], self.position[1])
            self.intention_position = self.model.nprandom.choice(neighbors)

    
    def bfs(self, start, goal):
        """ Implements BFS to find the path from start to goal """
        from collections import deque
        queue = deque([(start, [])])
        visited = set()
        
        while queue:
            current, path = queue.popleft()
            
            # Encontramos un camino
            if tuple(current) == tuple(goal):  
                return path + [current]
            
            if tuple(current) in visited:
                continue
            visited.add(tuple(current))
            
            for neighbor in self.model.get_neighbors(current[0], current[1]):
                if self.model.is_valid_position(neighbor) and tuple(neighbor) not in visited and (self.internal_board[neighbor[0]][neighbor[1]] == 0 or np.array_equal(neighbor, goal)): # Checo en mi internal_board
                    queue.append((neighbor, path + [current]))
        return []  # No econtramos camino
    
    def find_unvisited_path(self, start):
        """ BFS para encontrar a la celda no visitada mas cercana """
        from collections import deque
        queue = deque([(start, [])])
        visited = set()
        
        while queue:
            current, path = queue.popleft()
            
            # Encontramos un camino
            if not self.visited_cells[current[0]][current[1]] and current not in self.discharge_locations:  
                return path + [current]
            
            if tuple(current) in visited:
                continue
            visited.add(tuple(current))
            
            for neighbor in self.model.get_neighbors(current[0], current[1]):
                if self.model.is_valid_position(neighbor) and tuple(neighbor) not in visited and self.internal_board[neighbor[0]][neighbor[1]] == 0: # Checo en mi internal_board
                    queue.append((neighbor, path + [current]))
        return []  # No econtramos camino
        
    def visit_nearest_unvisited(self):
        if not self.unvisited_path:
            self.unvisited_path = self.find_unvisited_path(self.position)
            if self.unvisited_path:
                    self.unvisited_path.pop(0)
        
        if self.unvisited_path:
            # Nos intentamos mover a la posicion siguiente
            self.intention_position = self.unvisited_path.pop(0)
            x, y = self.intention_position
            self.intention = "move"
            if not self.model.is_valid_move_position(self.intention_position):
                # self.intention = "wait"
                # si no es valido hacemos movimiento random      
                self.intention = "move"
                neighbors = self.model.get_neighbors(self.position[0], self.position[1])
                self.intention_position = self.model.nprandom.choice(neighbors)
                self.unvisited_path = []
        else:
            self.intention = "wait"
    
    def chase_box(self):
         if not self.seen_box_path:
             start = self.position
             self.seen_box_path = self.bfs(start, random.choice(self.seen_box_stack))
             if self.seen_box_path:
                self.seen_box_path.pop(0)

         if self.seen_box_path:
            # Nos intentamos mover a la posicion siguiente
            self.intention_position = self.seen_box_path.pop(0)
            x, y = self.intention_position
            self.intention = "move"
            if not self.model.is_valid_move_position(self.intention_position):
                self.intention = "wait"
                self.seen_box_path = []
         else:
            self.intention = "wait"
            
    def explore(self):
        """ Explora el entorno """
        self.intention = "wait"  # Por defecto, espera
        if not self.has_box and self.nearby_single_boxes:
            self.unvisited_path = []
            self.intention = "pick_up"
            self.intention_position = self.model.nprandom.choice(self.nearby_single_boxes)
        else:
            if self.seen_box_stack:
                self.chase_box()
            else:
                self.visit_nearest_unvisited()
    
    def next(self):
        """ Determina el siguiente paso """
        plan = self.choose_plan()
        if plan == "find_discharge_location":
            self.find_discharge_location()
            self.unvisited_path = []
            self.seen_box_path = []
        elif plan == "explore":
            self.explore()
    
    def action(self):
        """ Ejecuta la acción """
        if self.intention == "stack":
            self.stack_boxes(*self.intention_position)
        elif self.intention == "pick_up":
            self.pick_box(*self.intention_position)
        elif self.intention == "move" and self.model.is_valid_move_position(self.intention_position):
            self.position = self.intention_position
    
    def move(self):
        """ Ejecuta un movimiento """
        if not self.model.is_floor_organized:
            self.model.total_moves += 1
        self.see()
        self.next()
        self.action()
        self.model.record_move(self)
    
    def pick_box(self, x, y):
        """ Recoge una caja """
        self.updateLookingDirection(x, y)
        if self.model.update_box_count(x, y, -1):
            self.has_box = True
        else:
            self.intention = "wait"
    
    def stack_boxes(self, x, y):
        """ Apila una caja """
        self.updateLookingDirection(x, y)
        if self.model.update_box_count(x, y, 1):
            self.has_box = False
        else:
            self.intention = "wait"
    
    def updateLookingDirection(self, to_x, to_y):
        """ Actualiza la dirección de visión del agente """
        from_x, from_y = self.position
        if to_x > from_x:
            self.looking_direction = "down"
        elif to_x < from_x:
            self.looking_direction = "up"
        elif to_y > from_y:
            self.looking_direction = "right"
        elif to_y < from_y:
            self.looking_direction = "left"


In [ ]:
class OscarAgent(RodrigoAgent):
    """
    Las reglas de este agente son las siguientes:
        - Si tiene una caja, busca la descarga más cercana y se dirige a ella.
        - Si no tiene caja pero recuerda alguna caja cercana, se dirige a ella.
        - De lo contrario explora el entorno.
    
    La idea para explorar el entorno es: determinar las celdas más cercanas no visitadas.
    Dada esa lista de celdas, elegir el camino hacia la celda final con más vecinos desconocidos.
    """
    
    def find_unvisited_path(self, start):
        """ BFS para encontrar a la celda no visitada mas cercana """
        from collections import deque
        queue = deque([(start, [])])
        visited = set()
        
        while queue:
            # Considerar todas las celdas a distancia N
            level_size = len(queue)
            possible_paths = []
            for _ in range(level_size):
                current, path = queue.popleft()

                # Encontramos un camino
                if not self.visited_cells[current[0]][current[1]] and current not in self.discharge_locations:  
                    possible_paths.append(path + [current])

                if tuple(current) in visited:
                    continue
                visited.add(tuple(current))

                for neighbor in self.model.get_neighbors(current[0], current[1]):
                    if self.model.is_valid_position(neighbor) and tuple(neighbor) not in visited and self.internal_board[neighbor[0]][neighbor[1]] == 0: # Checo en mi internal_board
                        queue.append((neighbor, path + [current]))
                        
            
            # Si hay posibles caminos, elegir la celda final que tiene más vecinos desconocidos
            if len(possible_paths) > 0:
                choose_index = 0
                greatest_unknown = self.unkown_neighbours(possible_paths[0][-1])
                
                for i in range(1, len(possible_paths)):
                    unknown = self.unkown_neighbours(possible_paths[i][-1])
                    if unknown > greatest_unknown:
                        greatest_unknown = unknown
                        choose_index = i
                
                return possible_paths[choose_index]

        return []  # No encontramos camino

    def unkown_neighbours(self, position):
        unkown = 0
        for neighbor in self.model.get_neighbors(position[0], position[1]):
            if self.model.is_valid_position(neighbor) and not self.visited_cells[neighbor[0]][neighbor[1]]:
                unkown += 1
        return unkown


In [ ]:
class PepeAgent(RodrigoAgent):
    """
    PepeAgent modifica la estrategia de exploración con las siguientes diferencias:
    -Prioriza explorar áreas con posibles pilas de cajas
    -Utiliza un enfoque más agresivo para buscar cajas
    -Implementa un método diferente de selección de celdas no visitadas """
    
    def find_unvisited_path(self, start):
        """ 
        BFS modificado para priorizar caminos cerca de posibles ubicaciones de cajas 
        y áreas con menos vecinos visitados
        """
        from collections import deque
        queue = deque([(start, [])])
        visited = set()
        possible_paths = []
        
        while queue:
            current, path = queue.popleft()
            
            # Priorizar caminos cerca de posibles ubicaciones de cajas o con menos vecinos visitados
            if (not self.visited_cells[current[0]][current[1]] and 
                current not in self.discharge_locations):
                # Puntuar el camino basado en la proximidad a posibles ubicaciones de cajas
                box_proximity_score = self.calculate_box_proximity(current)
                unvisited_neighbors_score = self.count_unvisited_neighbors(current)
                
                possible_paths.append((path + [current], box_proximity_score + unvisited_neighbors_score))
            
            if tuple(current) in visited:
                continue
            visited.add(tuple(current))
            
            for neighbor in self.model.get_neighbors(current[0], current[1]):
                if (self.model.is_valid_position(neighbor) and 
                    tuple(neighbor) not in visited and 
                    self.internal_board[neighbor[0]][neighbor[1]] == 0):
                    queue.append((neighbor, path + [current]))
        
        # Si se encuentran caminos, seleccionar basado en la puntuación combinada
        if possible_paths:
            return max(possible_paths, key=lambda x: x[1])[0]
        
        return []  # No se encontró camino
    
    def calculate_box_proximity(self, position):
        """ 
        Calcular una puntuación basada en la proximidad a posibles ubicaciones de cajas 
        y pilas de cajas conocidas
        """
        proximity_score = 0
        for box_location in self.seen_box_stack + self.nearby_box_stack:
            # Calcular la distancia de Manhattan
            distance = abs(position[0] - box_location[0]) + abs(position[1] - box_location[1])
            # Puntuación inversa de la distancia (ubicaciones más cercanas obtienen puntuaciones más altas)
            proximity_score += max(0, 10 - distance)
        
        return proximity_score
    
    def count_unvisited_neighbors(self, position):
        """ 
        Contar celdas vecinas no visitadas para fomentar la exploración 
        de áreas menos exploradas
        """
        unvisited_count = 0
        for neighbor in self.model.get_neighbors(position[0], position[1]):
            if (self.model.is_valid_position(neighbor) and 
                not self.visited_cells[neighbor[0]][neighbor[1]]):
                unvisited_count += 1
        
        return unvisited_count * 2  # Multiplicar para aumentar el peso
    
    def explore(self):
        """ 
        Estrategia de exploración modificada con búsqueda de cajas más agresiva 
        y selección inteligente de celdas
        """
        self.intention = "wait"  # Por defecto, esperar
        
        # Priorizar recoger cajas individuales de manera más agresiva
        if not self.has_box and self.nearby_single_boxes:
            self.unvisited_path = []
            self.intention = "pick_up"
            # Elegir la caja más cercana a posibles ubicaciones de descarga
            self.intention_position = min(
                self.nearby_single_boxes,
                key=lambda pos: self.calculate_discharge_distance(pos)
            )
        else:
            # Si no hay cajas individuales, seguir la estrategia de exploración modificada
            if self.seen_box_stack:
                # Persecución de cajas más estratégica
                self.chase_strategic_box()
            else:
                # Usar el método modificado para encontrar celdas no visitadas
                self.visit_nearest_unvisited()
    
    def calculate_discharge_distance(self, position):
        """ 
        Calcular la distancia a la ubicación de descarga más cercana 
        desde una posición dada
        """
        return min(
            ((position[0] - loc[0])**2 + (position[1] - loc[1])**2)**0.5
            for loc in self.discharge_locations
        )
    
    def chase_strategic_box(self):
        """ 
        Un enfoque más estratégico para perseguir cajas 
        """
        if not self.seen_box_path:
            start = self.position
            # Elegir la caja con la mejor posición estratégica
            strategic_box = max(
                self.seen_box_stack, 
                key=lambda box: (
                    self.calculate_box_proximity(box),
                    self.count_unvisited_neighbors(box)
                )
            )
            self.seen_box_path = self.bfs(start, strategic_box)
            if self.seen_box_path:
                self.seen_box_path.pop(0)
        
        if self.seen_box_path:
            # Intentar moverse a la siguiente posición
            self.intention_position = self.seen_box_path.pop(0)
            x, y = self.intention_position
            self.intention = "move"
            
            # Manejar posiciones de movimiento no válidas
            if not self.model.is_valid_move_position(self.intention_position):
                self.intention = "move"
                neighbors = self.model.get_neighbors(self.position[0], self.position[1])
                self.intention_position = self.model.nprandom.choice(neighbors)
                self.seen_box_path = []
        else:
            self.intention = "wait"

In [ ]:
class HectorAgent(RodrigoAgent):
    """
    agente híbrido que combina:
    
    componente reactivo:
    - responde inmediatamente a cajas cercanas
    - evita obstáculos
    - reacciona a puntos de descarga disponibles
    
    componente deliberativo:
    - mantiene mapa de densidad para exploración informada
    - calcula prioridades de cajas
    - planifica rutas usando bfs
    """
    
    def setup(self):
        super().setup()
        self.box_density = [[0.0 for _ in range(self.model.p.m)] 
                           for _ in range(self.model.p.n)]
        self.exploration_rate = 0.8  # comenzamos con alta exploración
        self.last_success = 0  # timestamp del último éxito con una caja
        
    def calculate_box_priority(self, pos):
        """
        evalúa el valor de una posición de caja basado en:
        - distancia al punto de descarga disponible más cercano
        - potencial para apilar
        - costo total de movimiento
        """
        x, y = pos
        
        # encuentra el punto de descarga más cercano con espacio
        discharge_dist = min(
            abs(x - dx) + abs(y - dy) 
            for dx, dy in self.discharge_locations
            if self.model.box_count[dx][dy] < 5
        )
        
        stack_potential = sum(
            1 for nx, ny in self.model.get_neighbors(x, y)
            if self.model.is_valid_position((nx, ny)) 
            and 0 < self.model.box_count[nx][ny] < 5
        )
        
        # considera el costo total de movimiento
        movement_cost = abs(x - self.position[0]) + abs(y - self.position[1])
        
        # un puntaje más bajo es mejor
        return (discharge_dist + movement_cost) / (1 + 2 * stack_potential)

    def update_density_map(self):
        """
        actualiza la memoria del agente sobre la distribución de cajas
        usando un decay para olvidar poco a poco observaciones viejas
        """
        decay = 0.95
        for i in range(self.model.p.n):
            for j in range(self.model.p.m):
                self.box_density[i][j] *= decay
                if self.model.box_count[i][j] > 0:
                    self.box_density[i][j] = 1.0
                    
                    for ni, nj in self.model.get_neighbors(i, j):
                        if self.model.is_valid_position((ni, nj)):
                            self.box_density[ni][nj] = max(
                                self.box_density[ni][nj],
                                0.7  
                            )

    def find_best_discharge(self):
        """
        identifica el punto de descarga más estratégico considerando:
        - capacidad actual
        - distancia
        - potencial para apilar
        """
        valid_locations = [
            loc for loc in self.discharge_locations
            if self.model.box_count[loc[0]][loc[1]] < 5
        ]
        
        if not valid_locations:
            return None
            
        return min(
            valid_locations,
            key=lambda loc: (
                abs(loc[0] - self.position[0]) + 
                abs(loc[1] - self.position[1])
            )
        )

    def explore(self):
        # actualizamos nuestro conocimiento
        self.update_density_map()
        
        if not self.has_box and self.nearby_single_boxes:
            # selección de la caja
            self.intention = "pick_up"
            self.intention_position = min(
                self.nearby_single_boxes,
                key=self.calculate_box_priority
            )
            self.last_success = self.model.t
            
        elif self.seen_box_stack:
            # persigue la mejor caja recordada
            target = min(
                self.seen_box_stack,
                key=lambda pos: (
                    self.calculate_box_priority(pos) * 
                    (1 + abs(pos[0] - self.position[0]) + 
                     abs(pos[1] - self.position[1]))
                )
            )
            path = self.bfs(self.position, target)
            if path and len(path) > 1:
                self.intention = "move"
                self.intention_position = path[1]
            else:
                self.explore_adaptively()
        else:
            self.explore_adaptively()

    def explore_adaptively(self):
        unvisited = [
            (i, j) for i in range(self.model.p.n) 
            for j in range(self.model.p.m)
            if not self.visited_cells[i][j] and 
               self.model.is_valid_position((i, j))
        ]
        
        if unvisited:
            # balancea distancia y potencial
            target = min(
                unvisited,
                key=lambda pos: (
                    (abs(pos[0] - self.position[0]) + 
                     abs(pos[1] - self.position[1])) *
                    (1 - self.box_density[pos[0]][pos[1]])
                )
            )
            path = self.bfs(self.position, target)
            if path and len(path) > 1:
                self.intention = "move"
                self.intention_position = path[1]
            else:
                self.default_movement()
        else:
            self.default_movement()

    def default_movement(self):
        self.intention = "move"
        valid_neighbors = [
            n for n in self.model.get_neighbors(*self.position)
            if self.model.is_valid_move_position(n)
        ]
        if valid_neighbors:
            # elige dirección hacia áreas de mayor densidad histórica
            self.intention_position = max(
                valid_neighbors,
                key=lambda pos: self.box_density[pos[0]][pos[1]]
            )
        else:
            self.intention = "wait"

In [ ]:
class SergioAgent(ap.Agent):
    """
    Las reglas de este agente son las siguientes:
        - Si encuentra una caja a su alrededor, la agarra 
        - Si tiene una caja y está cerca del centro, la apila
        - Si tiene una caja, encuentra el mejor camino al centro y se dirige en esa direccion
        - De lo contrario, explora las columnas una por una, de arriba a abajo, eligiendo la mas cercana no explorada en su totalidad
    """
    
    def setup(self):
        self.intention = None  # Intención actual (pick_up, stack, move o wait)
        self.intention_position = None  # Posición objetivo al hacer acción
        
        self.nearby_single_boxes = []  # Coordenadas de las cajas cercanas (stacks de 1 sola caja)
        self.nearby_box_stack = []  # Coordenadas de stacks de cajas cercanos (stacks de más de 1 caja)
        self.has_box = False
        
        self.looking_direction = "Up"  # Dirección en la que está mirando
        
        # Inicializar tablero interno
        self.internal_board = [
            [0 for _ in range(self.model.p.m)] for _ in range(self.model.p.n)
        ]
        
        self.bfs_path = []  # Lista de movimientos para el BFS
        self.discharge_locations = self.model.discharge_locations
        self.discharge_location = (-1, -1) # al principio no hay ninguna seleccionada
        
        self.visited_cells = [
            [False for _ in range(self.model.p.m)] for _ in range(self.model.p.n)
        ]
        self.visited_column = [0 for _ in range(self.model.p.m)] # Para verificar más rápido si todas las celdas de una columna han sido visitadas
        for x, y in self.discharge_locations:
            self.visited_cells[x][y] = True
            

    def setup_position(self, start_position):
        self.position = start_position  # Posición inicial del agente
        
    def see(self):
        """ Percibe su entorno y actualiza el internal_board """
        x, y = self.position
        self.nearby_single_boxes = []
        self.nearby_box_stack = []
        self.visited_cells[x][y] = True
        
        neighbors = self.model.get_neighbors(x, y)
        for neighbor_pos in neighbors:
            i, j = neighbor_pos
            if not self.model.is_valid_position(neighbor_pos):
                continue
            
            box_count = self.model.box_count[i][j]
            self.internal_board[i][j] = box_count
            
            if box_count == 1 and (i, j) not in self.discharge_locations:
                self.nearby_single_boxes.append(neighbor_pos)
            elif 1 <= box_count <= 4:
                self.nearby_box_stack.append(neighbor_pos)
            elif box_count == 0:
                self.visited_cells[i][j] = True # Uninteresting position
        
        # Update internal visited 
        for j in range(self.model.p.m):
            if self.visited_column[j]:
                continue
            
            sum = 0
            for i in range(self.model.p.n):
                sum += self.visited_cells[i][j]
            if sum == self.model.p.n:
                self.visited_column[j] = True
            

    
    def next(self):
        """ Determina el siguiente paso """
        plan = self.choose_plan()
        if plan == "find_discharge_location":
            self.find_discharge_location()
        elif plan == "explore":
            self.explore()
    
    def action(self):
        """ Ejecuta la acción """
        if self.intention == "stack":
            self.stack_boxes(*self.intention_position)
        elif self.intention == "pick_up":
            self.pick_box(*self.intention_position)
        elif self.intention == "move" and self.model.is_valid_move_position(self.intention_position):
            self.position = self.intention_position
    
    def move(self):
        """ Ejecuta un movimiento """
        if not self.model.is_floor_organized:
            self.model.total_moves += 1
        self.see()
        self.next()
        self.action()
        self.model.record_move(self)
    
    
    """ Explore related functions"""
    
    def explore(self):
        """ Explora el entorno """
        self.intention = "wait"  # Por defecto, espera
        if not self.has_box and self.nearby_single_boxes:
            self.intention = "pick_up"
            self.intention_position = self.model.nprandom.choice(self.nearby_single_boxes)
        else:
            if not self.bfs_path:
                self.find_objective()
            
            self.follow_bfs_path()
    
    def find_objective(self): 
        # Start at impossible value
        columns = [ j for j in range(self.model.p.m) if not self.visited_column[j] ]
        if len(columns) == 0:
            self.intention = "wait"
            return
        
        objective_column = min(
            columns,
            key= lambda j: abs(self.position[1] - j)
        )
        
        objective_row = -1
        for i in range(self.model.p.m):
            if not self.visited_cells[i][objective_column]:
                objective_row = i
                break
        
        if objective_row == -1:
            self.intention = "wait"
            return
        
        # Find best path to that cell with bfs
        self.bfs_path = self.bfs(self.position, (objective_row, objective_column))
        if self.bfs_path:
            self.bfs_path.pop(0)
        
        
    """ Aux functions"""
    
    def choose_plan(self):
        """ Escoge el plan según el estado actual """
        if self.has_box:
            return "find_discharge_location"
        else:
            return "explore"
    
    def find_discharge_location(self):
        """ Encuentra la ruta a una discharge_location usando BFS """
        # Vemos si alguno de los vecinos es la discharge_location que buscamos
        for neighbor in self.model.get_neighbors(*self.position):
            
            if (int(neighbor[0]), int(neighbor[1])) == (int(self.discharge_location[0]), int(self.discharge_location[1])):
                if self.model.box_count[neighbor[0]][neighbor[1]] < 5:
                    self.intention = "stack"
                    self.intention_position = neighbor
                    
                    # checamos si hay un agente en la posicion donde queremos poner una caja que es adjacente a nosotros  
                    for agent in self.model.all_agents:
                        if np.array_equal(agent.position, self.intention_position):  # Si pasa hacemos un movimiento random
                            self.intention = "move"
                            neighbors = self.model.get_neighbors(self.position[0], self.position[1])
                            self.intention_position = self.model.nprandom.choice(neighbors)
                    self.bfs_path = []
                    return
                else:
                    self.bfs_path = []
            

        # Si no hemos computado el mejor camino al centro lo encontramos con bfs
        if not self.bfs_path:
            start = self.position
            # Calculamos el centro del tablero
            center_x, center_y = self.p.n / 2, self.p.m / 2

            # Filtramos las locaciones invalidas
            valid_discharge_locations = [
                loc for loc in self.discharge_locations
                if self.internal_board[loc[0]][loc[1]] < 5  
            ]

            # Seleccionamos la mas cercana al centro
            if valid_discharge_locations:
                self.discharge_location = min(
                    valid_discharge_locations,
                    key=lambda loc: ((loc[0] - center_x) ** 2 + (loc[1] - center_y) ** 2) ** 0.5
                )
            else:
                return
            # Encontramos el mejor camino
            self.bfs_path = self.bfs(start, self.discharge_location)
            if self.bfs_path:
                self.bfs_path.pop(0)
        
        self.follow_bfs_path()
    
    def follow_bfs_path(self):
        if self.bfs_path:
            # Nos intentamos mover a la posicion siguiente
            self.intention_position = self.bfs_path.pop(0)
            x, y = self.intention_position
            self.intention = "move"
            if not self.model.is_valid_move_position(self.intention_position):
                # Hacemos movimiento random
                self.intention = "move"
                neighbors = self.model.get_neighbors(self.position[0], self.position[1])
                self.intention_position = self.model.nprandom.choice(neighbors)
                self.bfs_path = []
        else:
            self.intention = "move"
            neighbors = self.model.get_neighbors(self.position[0], self.position[1])
            self.intention_position = self.model.nprandom.choice(neighbors)
    
    def bfs(self, start, goal):
        """ Implements BFS to find the path from start to goal """
        from collections import deque
        queue = deque([(start, [])])
        visited = set()
        
        while queue:
            current, path = queue.popleft()
            
            # Encontramos un camino
            if tuple(current) == tuple(goal):  
                return path + [current]
            
            if tuple(current) in visited:
                continue
            visited.add(tuple(current))
            
            for neighbor in self.model.get_neighbors(current[0], current[1]):
                if self.model.is_valid_position(neighbor) and tuple(neighbor) not in visited and (self.internal_board[neighbor[0]][neighbor[1]] == 0 or np.array_equal(neighbor, goal)): # Checo en mi internal_board
                    queue.append((neighbor, path + [current]))
        return []  # No econtramos camino
    
    def pick_box(self, x, y):
        """ Recoge una caja """
        self.updateLookingDirection(x, y)
        if self.model.update_box_count(x, y, -1):
            self.has_box = True
            self.visited_cells[x][y] = True # Uninteresting position now
            self.bfs_path = [] # Reset path
        else:
            self.intention = "wait"
    
    def stack_boxes(self, x, y):
        """ Apila una caja """
        self.updateLookingDirection(x, y)
        if self.model.update_box_count(x, y, 1):
            self.has_box = False
            self.bfs_path = [] # Reset path
        else:
            self.intention = "wait"
    
    def updateLookingDirection(self, to_x, to_y):
        """ Actualiza la dirección de visión del agente """
        from_x, from_y = self.position
        if to_x > from_x:
            self.looking_direction = "down"
        elif to_x < from_x:
            self.looking_direction = "up"
        elif to_y > from_y:
            self.looking_direction = "right"
        elif to_y < from_y:
            self.looking_direction = "left"


In [ ]:
class BoxWarehouseModel(ap.Model):
    """ Modelo de limpieza de tablero """
    
    def record_move(self, agent):
        move = Move(
            agent_id=agent.id,
            action=agent.intention,
            cell=agent.position,
            looking_direction=agent.looking_direction
        )
        move.add_move()
    
    def setup(self):
        self.box_positions = None
        self.box_count, self.agent_positions, self.discharge_locations = self.generateFloorPlan(self.p.total_boxes)
        
        self.random_agents = ap.AgentList(self, self.p.random_agents, RandomAgent)
        self.rodrigo_agents = ap.AgentList(self, self.p.rodrigo_agents, RodrigoAgent)
        self.sergio_agents = ap.AgentList(self, self.p.sergio_agents, SergioAgent)
        self.hector_agents = ap.AgentList(self, self.p.hector_agents, HectorAgent)
        self.pepe_agents = ap.AgentList(self, self.p.pepe_agents, PepeAgent)
        self.oscar_agents = ap.AgentList(self, self.p.oscar_agents, OscarAgent)
        
        self.all_agents = self.random_agents  + self.rodrigo_agents + self.sergio_agents + self.hector_agents + self.pepe_agents + self.oscar_agents# be careful, there can only be 5 of them
        for idx, agent in enumerate(self.all_agents):
            if idx > len(self.agent_positions): 
                raise RuntimeError("There cannot be more than 5 agents in total!")
            agent.setup_position(self.agent_positions[idx])

        
        # Variables para generar estadisticas
        self.total_moves = 0
        self.is_floor_organized = True if self.p.total_boxes <= 0 else False # Es falso a menos que no haya cajas
        self.finish_time = self.p.steps + 1 # Por default no termina en tiempo
        
        
        # Creamos la posicion inicial
        self.initial_position = InitialPosition(
            agents=[(agent.id, agent.position) for agent in self.all_agents],
            board_dimensions=(self.p.n, self.p.m),
            box_positions=self.box_positions,
            discharge_positions=self.discharge_locations
        )

        # Empezamos a guardar los diferentes movimientos
        Move.start_new_game()
        self.rounds = None
        
    def step(self):
        Move.start_new_round()
        self.all_agents.move()

    def update(self):
        pass

    def end(self):
        
        self.report('total_moves', self.total_moves)
        self.report('time_taken', self.finish_time)
        self.report('percentage_time_taken', self.finish_time / self.p.steps)
        self.rounds = Move.rounds
    
    
    """ Aux functions """
 
    def generateFloorPlan(self, k):
        """
        Generates an n x m floor plan with k boxes, 5 agents, and a designated 
        discharge location square at the center. Boxes and agents will not spawn 
        in the discharge locations.
        
        Parameters:
            k (int): Number of boxes.
        """
        discharge_side = self.p.discharge_side
        # Initialize the matrix with zeros
        box_count = [[0 for _ in range(self.p.m)] for _ in range(self.p.n)]
        
        # Compute discharge locations
        center_x = self.p.n // 2
        center_y = self.p.m // 2
        half_side = discharge_side // 2
        
        discharge_locations = [
            (i, j)
            for i in range(center_x - half_side, center_x + half_side + discharge_side % 2)
            for j in range(center_y - half_side, center_y + half_side + discharge_side % 2)
        ]
        
        # Generate all possible positions excluding discharge locations
        all_positions = [
            (i, j) for i in range(self.p.n) for j in range(self.p.m)
            if (i, j) not in discharge_locations
        ]
        
        # Too many boxes and agents for the floor size
        if k + 5 > len(all_positions):
            raise RuntimeError("Too many boxes for the floor size")
        
        # Select positions for boxes and agents
        unique_positions: list = self.nprandom.choice(all_positions, k + 5, replace=False)
        self.box_positions = unique_positions[:-5]  # All elements except the last 5
        agent_positions = unique_positions[-5:]    # Only the last 5 elements
        
        # Mark the selected positions for boxes
        for i, j in self.box_positions:
            box_count[i][j] = 1
        
        return [box_count, agent_positions, discharge_locations]

    def is_valid_position(self, position):
        """ Verifica si una posición es valida dentro de nuestro tablero """
        x, y = position
        return 0 <= x < self.p.n and 0 <= y < self.p.m

    def is_valid_move_position(self, position):
        """ Verifica si una posición es valida dentro de nuestro tablero """
        x, y = position
        if not (0 <= x < self.p.n and 0 <= y < self.p.m):
            # print("lost 1")
            return False
        
        for agent in self.all_agents:
            if agent.position[0] == x and agent.position[1] == y:
                # print("lost 2")
                return False
        
        if self.box_count[x][y] > 0:
            # print("lost 3")
            return False
        
        return True
    
    def get_neighbors(self, x, y):
        """ regresa lista con todas las celdas vecinas de la posicion x, y"""
        neighbors = []
        
        for dx in range(-1, 2):  # Iterata -1, 0, 1
            for dy in range(-1, 2):  # Itera -1, 0, 1
                # Saltamos la celda actual y diagonales, robots solo ven en direcciones cardinales
                if abs(dx) == abs(dy): 
                    continue
                neighbors.append((x + dx, y + dy)) 
    
        return neighbors

    def update_box_count(self, x, y, dif):
        """ 
        Suma dif a self.box_count[x][y] y verifica si se terminaron de organizar las cajas.
        EN ESTE MOMENTO, organizar las cajas significa que todas las cajas esten en una pila de al menos 2 cajas
        Return True si realizo la operacion correctamente, False si no
        """
        # Not a valid move
        if self.box_count[x][y] + dif < 0 or self.box_count[x][y] + dif > 5:
            return False
        
        # Verificar si algún agente está en la posición (x, y)
        for agent in self.model.all_agents:
            if np.array_equal(agent.position, [x, y]):  # Cambiar `agent.position` a la estructura adecuada si es necesario
                return False
            
        self.box_count[x][y] += dif
 
        # Si todas las discharge_position estan llenas o todas las cajas estan en discharge_positions consideramos que ganamos
        full_discharge_positions = True
        discharged_boxes = 0
        for discharge_location in self.discharge_locations:
            x, y = discharge_location
            discharged_boxes += self.box_count[x][y]
            if self.box_count[x][y] != 5:
                full_discharge_positions = False
        
        
        self.is_floor_organized = (full_discharge_positions or (self.p.total_boxes == discharged_boxes))
        # Termina la simulacion cuando se organizan todas las cajas
        if self.is_floor_organized:
            self.finish_time = self.t
            self.stop()
            
        return True


In [ ]:
def multirun_experiment():
    # ciudado! Se usara 2 ** simulations_log_2. No hagas este valor muy alto
    simulations_log_2 = 2
    iterations_per_sample = 2 
    parameters = {
        'steps': ap.IntRange(20*20, 20*20*3),
        'n': 20,
        'm': 20,
        'total_boxes': ap.IntRange(10, 80),
        'discharge_side': 4,
        'sergio_agents': 0,
        'rodrigo_agents': 0,
        'pepe_agents': 0,
        'oscar_agents': 0,
        'hector_agents': 0,
        'random_agents': 0,
        'seed': 22
    }

    agent_types = [ 
                   'hector_agents',
                    'sergio_agents',
                    'rodrigo_agents',
                    'oscar_agents',
                    'pepe_agents'
                    ]

    runs = {}
    for agent in agent_types: 
        # if agent != 'sergio_agents':  # Uncomment to test only your agent
        #     continue
        
        # Asign valid values to current agent only
        for other_agent in agent_types:
            if agent == other_agent:
                parameters[other_agent] = 5
            else: 
                parameters[other_agent] = 0
                
        sample = ap.Sample(
            parameters,
            n= 2 ** simulations_log_2,
            method='saltelli',
            calc_second_order=False
        )
        exp = ap.Experiment(BoxWarehouseModel, sample, iterations=iterations_per_sample, record=True)
        results = exp.run()
        reporters_df = results.reporters
        
        curr_runs = {
            'A': 0,
            'B': 0,
            'C': 0,
            'D': 0,
            'not_finished': 0,
            'total': 0,
        }
        
        curr_runs['total'] = len(reporters_df)
        # print(reporters_df)
        
        curr_runs['A'] = (reporters_df['percentage_time_taken'] <= 0.25).sum() / curr_runs['total']
        curr_runs['B'] = (reporters_df['percentage_time_taken'] <= 0.50).sum() / curr_runs['total']
        curr_runs['C'] = (reporters_df['percentage_time_taken'] <= 0.75).sum() / curr_runs['total']
        curr_runs['D'] = (reporters_df['percentage_time_taken'] <= 1.00).sum() / curr_runs['total']

        curr_runs['not_finished_probability'] = (reporters_df['percentage_time_taken'] > 1.00).sum() / curr_runs['total']
        
        curr_runs['A'] = round(curr_runs['A'], 3)
        curr_runs['B'] = round(curr_runs['B'], 3)
        curr_runs['C'] = round(curr_runs['C'], 3)
        curr_runs['D'] = round(curr_runs['D'], 3)
        curr_runs['not_finished_probability'] = round(curr_runs['not_finished_probability'], 3)
        runs[agent] = curr_runs
        
    
    return runs
        

In [ ]:
def show_animation(parameters=None):
    if parameters is None:
        parameters = {
            'steps': 1000,
            'n': 20,
            'm': 20,
            'total_boxes': 80,
            'discharge_side': 4,
            'sergio_agents': 5,
            'rodrigo_agents': 0,
            'pepe_agents': 0,
            'oscar_agents': 0,
            'hector_agents': 0,
            'random_agents': 0,
            'seed': 22
        }

    def animation_plot(model, ax):
        color_dict = {
            0: '#000000',  # Black for empty cells
            1: '#006600',  # Dark green for cells with boxes
            2: '#FF0000',  # Red for agents
            3: '#808080'   # Gray for discharge locations
        }
        
        # Create base visualization array
        ndarray_doubles = np.zeros_like(model.box_count, dtype=float)
        
        # Mark cells with boxes
        for i in range(len(model.box_count)):
            for j in range(len(model.box_count[0])):
                if model.box_count[i][j] > 0:
                    ndarray_doubles[i][j] = 1
        
        # Mark discharge locations
        for x, y in model.discharge_locations:
            ndarray_doubles[x][y] = 3
            
        # Mark agent positions
        for agent in model.all_agents:
            x, y = agent.position
            ndarray_doubles[x][y] = 2
        
        # Plot the base grid
        ap.gridplot(ndarray_doubles, ax=ax, color_dict=color_dict, convert=True)
        
        # Add text annotations for box counts
        max_dim = max(parameters['n'], parameters['m'])
        font_size = min(-1/6 * max_dim + 13.33, 10)
        font_size = font_size if font_size >= 4 else 0  # Just don't show if it's too small

        for i in range(len(model.box_count)):
            for j in range(len(model.box_count[0])):
                if model.box_count[i][j] > 0:
                    # Add text with white color for better visibility on dark green
                    ax.text(j, i, str(model.box_count[i][j]), 
                        ha='center', va='center', color='white',
                        fontweight='bold', fontsize=font_size)
        
        ax.set_title(f"Box Warehouse Model\nTime-step: {model.t}")
        
    fig, ax = plt.subplots()
    model = BoxWarehouseModel(parameters)
    animation = ap.animate(model, fig, ax, animation_plot)
    return animation


In [ ]:
# Grafica A, B, C, D
def plot_results(runs):
    categories = ['A', 'B', 'C', 'D']
    agents = list(runs.keys())
    data = {category: [runs[agent][category] for agent in agents] for category in categories}

    x = np.arange(len(agents))
    width = 0.2  

    fig, ax = plt.subplots(figsize=(10, 6))  
    for i, category in enumerate(categories):
        ax.bar(x + i * width, data[category], width, label=category)

    ax.set_xlabel('Agent Types')
    ax.set_ylabel('Proportion')
    ax.set_title('Comparison of Results by Agent Type')
    ax.set_xticks(x + width * (len(categories) - 1) / 2)
    ax.set_xticklabels([name.split('_')[0].capitalize() for name in agents], rotation=45, ha='right')  
    ax.legend()

    plt.tight_layout()  
    plt.show()


In [ ]:
options = {
    'show_animation': False,
    'multirun_experiment': False, # Tested
    'single_experiment': False, # Tested 
} # Estas siempre tienen que estar en falso a menos que estemos probando algo porque tambien se llaman desde PlanoCarros

In [ ]:
if options['show_animation']:
    animation = show_animation()
    display(
        HTML(animation.to_jshtml())
    )    

In [ ]:
if options['multirun_experiment']:
    results = multirun_experiment()
    plot_results(results)
    print(results)

In [ ]:
if options["single_experiment"]: # No borrar
    parameters = {
        'steps': 100,
        'n': 20,
        'm': 20,
        'total_boxes': 250,
        'discharge_side': 4,
        'sergio_agents':  0,
        'rodrigo_agents': 0,
        'pepe_agents': 0,
        'oscar_agents': 1,
        'hector_agents': 0,
        'random_agents': 0,
        # 'seed': 22,
    }
    model = BoxWarehouseModel(parameters)
    results = model.run()